### Importing libraries

In [3]:
from azure.cosmos import CosmosClient
from azure.core.credentials import AzureKeyCredential
from azure.identity import ClientSecretCredential, DefaultAzureCredential
from azure.cosmos.partition_key import PartitionKey

from dotenv import load_dotenv
import pandas as pd
load_dotenv(override=True)
import json
import pyodbc

import os

# Using DefaultAzureCredential (recommended)
# https://techcommunity.microsoft.com/t5/azure-architecture-blog/configure-rbac-for-cosmos-db-with-managed-identity-instead-of/ba-p/3056638#:~:text=Create%20custom%20roles%20MyReadOnlyRole%20and%20MyReadWriteRole%20with%20both,definition%20create%20-a%20%24accountName%20-g%20%24resourceGroupName%20-b%20%40role-definition-ro.json
aad_credentials = DefaultAzureCredential()

AZURE_COSMOS_DB_ENDPOINT=os.environ['AZURE_COSMOS_DB_ENDPOINT']
AZURE_COSMOS_DB_KEY= os.environ['AZURE_COSMOS_DB_KEY']
AZURE_COSMOS_DB_DATABASE= os.environ['AZURE_COSMOS_DB_DATABASE']
AZURE_COSMOS_DB_CONN= os.environ['AZURE_COSMOS_DB_CONN']
azurecosmosdbclient = CosmosClient(AZURE_COSMOS_DB_ENDPOINT, credential=aad_credentials)

CONTAINER_ID = os.environ['AZURE_COSMOS_DB_CONTAINER']
PartitionKeyPath = "/chunk_id"

database_client = azurecosmosdbclient.get_database_client(AZURE_COSMOS_DB_DATABASE)
container_client = database_client.get_container_client(CONTAINER_ID)

SQL_SERVER_USERNAME = os.environ["SQL_SERVER_USERNAME"]
SQL_SERVER_ENDPOINT = os.environ["SQL_SERVER_ENDPOINT"]
SQL_SERVER_PASSWORD = os.environ["SQL_SERVER_PASSWORD"]
SQL_SERVER_DATABASE = os.environ["SQL_SERVER_DATABASE"] 
driver = "{ODBC Driver 18 for SQL Server}"
sqlalchemy_driver = "ODBC Driver 18 for SQL Server"

## Creatng a connection to the SQL Server
conn = pyodbc.connect(f'DRIVER={driver};SERVER={SQL_SERVER_ENDPOINT};PORT=1433;DATABASE={SQL_SERVER_DATABASE};UID={SQL_SERVER_USERNAME};PWD={SQL_SERVER_PASSWORD}')  
cursor = conn.cursor()

from openai import AzureOpenAI
aoai_client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-07-01-preview"
)

### AzureSQLDB Tool calling

#### SQLDatabase GetSQLDBAnswer Function definition

In [ ]:
#https://github.com/Azure-Samples/openai/blob/main/End_to_end_Solutions/AOAISearchDemo/notebooks/structured_data_retreival_nltosql.ipynb

In [4]:
def GetSQLDBAnswer(question):

    prompt_template = """ You are a SQL programmer Assistant.Your role is to generate SQL code (SQL Server) to retrieve an answer to a natural language query. Make sure to disambiguate column names when creating queries that use more than one table. If a valid SQL query cannot be generated, only say "ERROR:" followed by why it cannot be generated.
                  Do not answer any questions on inserting or deleting rows from the table. Instead, say "ERROR: I am not authorized to make changes to the data"

                  Use the following sales database schema to write SQL queries:
                  Customers(cust_id INTEGER, cust_name VARCHAR, cust_email VARCHAR, cust_phone VARCHAR, cust_address VARCHAR, PRIMARY KEY (cust_id))
                  Products(prod_id INTEGER,prod_name varchar, price FLOAT, category VARCHAR, PRIMARY KEY(prod_id))
                  Stock(prod_id INTEGER, merchant_id INTEGER, stock INTEGER, PRIMARY KEY(prod_id, merchant_id), FOREIGN KEY(merchant_id, prod_id))
                  Merchants(merchant_id INTEGER, merchant_name VARCHAR, merchant_region VARCHAR, merchant_address VARCHAR, PRIMARY KEY(merchant_id))
                  Sales(sale_id INTEGER, cust_id INTEGER , merchant_id INTEGER , date TIMESTAMP, total_price FLOAT, PRIMARY KEY(sale_id),FOREIGN KEY(cust_id,merchant_id))
                  Sales_Detail(sales_id INTEGER, prod_id INTEGER, quantity INTEGER, PRIMARY KEY(sales_id,prod_id), FOREIGN KEY(sales_id,prod_id))

                  Examples:
                  User: List all Surface accessories, along with their prices. SQL Code:
                  Assistant: SELECT prod_name, category, price FROM Products WHERE prod_name like '%Surface%' and category like '%accessory%';
                  User: Which is the cheapest Surface device? SQL Code:
                  Assistant: SELECT TOP 1 prod_name, price FROM Products WHERE prod_name like '%Surface%' ORDER BY price ASC;
                  User: How many Surface Laptop 5 does GadgetWorld have?
                  Assistant: SELECT Merchants.merchant_id, Merchants.merchant_name, SUM(stock) as total_stock FROM Stock JOIN Merchants ON Stock.merchant_id = Merchants.merchant_id WHERE prod_id IN (SELECT prod_id FROM Products WHERE prod_name LIKE '%Surface Laptop 5%' and merchant_name like '%GadgetWorld%') GROUP BY Merchants.merchant_id, Merchants.merchant_name;
                  User: how many surface devices were sold last week?
                  Assistant: SELECT Sum(sales_detail.quantity) AS total_surface_devices_sold FROM sales_detail JOIN sales ON sales_detail.sales_id = sales.sale_id JOIN products ON sales_detail.prod_id = products.prod_id WHERE products.prod_name LIKE '%Surface%' AND sales.date >= Dateadd(wk, Datediff(wk, 0, Getdate()) - 1, 0) AND sales.date < Dateadd(wk, Datediff(wk, 0, Getdate()), 0);

                  ## Additional guidance
                    ** Please only answer questions that can be answered with the data provided. Do not make any assumptions about the data.
                    ** Please write the code for a SQL Server database.
                    ** If you need to ask any clafication, please ask for it in the format JSON format with the key: CLARIFICATION
                    ** If asked about any date, default to filing_date unless the question implies that is a question about the filing period.
                    ** Dates should be explicity stated in the query. For example, if you are asked about filings in 2023, you should use the following format: '2023-01-01' and '2023-12-31'. Otehrwise, ask for the specific period.
                    ** Provide the answer in a JSON format with the key: SQLquery
                  question: """
    
    messages = [{"role":"system","content":prompt_template}, 
               {"role":"user","content":question}]

    response = aoai_client.chat.completions.create(model="gpt4o",  
                                        messages = messages, 
                                        temperature=0.1,  
                                        max_tokens=2000,
                                        response_format={ "type": "json_object" },
                                        seed = 42)
    SQLschema = response.choices[0].message.content
    
    if "ERROR" in SQLschema:
        print('ERROR')
        return str({"SQLquery":"No query", "Answer": str(json.loads(SQLschema)['ERROR'])})
    elif 'CLARIFICATION' in SQLschema:
        print('CLARIFICATION')
        return str({"SQLquery":"No query", "Answer": str(json.loads(SQLschema)['CLARIFICATION'])})
    else:
        SQLQuery = json.loads(SQLschema)['SQLquery']
        print(SQLQuery)
        SQLResult = cursor.execute(SQLQuery)
        return str({"SQLquery":SQLQuery, "Answer": str([row for row in SQLResult.fetchall()])})

In [5]:
output = GetSQLDBAnswer("List all Surface accessories, along with their prices")
print(output)

SELECT prod_name, category, price FROM Products WHERE prod_name like '%Surface%' and category like '%accessory%';
{'SQLquery': "SELECT prod_name, category, price FROM Products WHERE prod_name like '%Surface%' and category like '%accessory%';", 'Answer': "[('Surface Slim Pen 2', 'Accessory', 94.99), ('Surface Pen', 'Accessory', 99.99), ('Surface Dock', 'Accessory', 199.99)]"}


In [6]:
def run_conversation(question):

    with open("../data/processed/tool_description/GetSQLDBAnswerDescription.txt", "r") as file:
        sql_tool_description = file.read()
    # Initial user message
    messages = [{"role":"system","content":"You are an assistant that help answering questions from either an AI Search Index or a CosmosDB Database. Please only answer the questions if any of the tools provide it, If tool_calls=None please state that you dont have access to the source of that information and pass the message from the function calls"}, {"role": "user", "content": question}] # Single function call

    # Define the function for the model
    tools = [
        {
            "type": "function",
            "function": {
                "name": "GetSQLDBAnswer",
                "description": sql_tool_description,
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "question that needs to be answered using the CosmosDB database. Please pass in verbatim the question that the user asked.",
                        },
                    },
                    "required": ["query"],
                },
            }
        }
    ]

    # First API call: Ask the model to use the function
    response = aoai_client.chat.completions.create(
        model="gpt4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        seed=42
    )

    # Process the model's response
    response_message = response.choices[0].message
    messages.append(response_message)


    print("Model's response:")  
    print(response_message.tool_calls)  

    # Handle function calls
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(f"Function call: {function_name}")  
            print(f"Function arguments: {function_args}") 

            if function_name == "GetSQLDBAnswer":
                function_response = GetSQLDBAnswer(
                    question=function_args.get("question")
                )
                print(f"Function response: {function_response}")
            else:
                function_response = "Function not found"
            
        messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
    else:
        print("No tool calls were made by the model.")
        messages.append({"role": "system", "content": "No functions or tools were called by the model."})
    
    
    print(messages)
    final_response = aoai_client.chat.completions.create(
        model='gpt4o',
        messages=messages,
        seed=42
    )

    print("--------------------------------------------------------------------------------")
    print("Final Response:")
    print(final_response.choices[0].message.content)
    print("--------------------------------------------------------------------------------")
    return final_response.choices[0].message.content

In [9]:
run_conversation("How many Surface Laptops were sold during 2023")

Model's response:
[ChatCompletionMessageToolCall(id='call_IIMq9yUEqKKOH9Mtidx7D2y4', function=Function(arguments='{"question":"How many Surface Laptops were sold during 2023"}', name='GetSQLDBAnswer'), type='function')]
Function call: GetSQLDBAnswer
Function arguments: {'question': 'How many Surface Laptops were sold during 2023'}
SELECT SUM(sales_detail.quantity) AS total_surface_laptops_sold FROM sales_detail JOIN sales ON sales_detail.sales_id = sales.sale_id JOIN products ON sales_detail.prod_id = products.prod_id WHERE products.prod_name LIKE '%Surface Laptop%' AND sales.date >= '2023-01-01' AND sales.date <= '2023-12-31';
Function response: {'SQLquery': "SELECT SUM(sales_detail.quantity) AS total_surface_laptops_sold FROM sales_detail JOIN sales ON sales_detail.sales_id = sales.sale_id JOIN products ON sales_detail.prod_id = products.prod_id WHERE products.prod_name LIKE '%Surface Laptop%' AND sales.date >= '2023-01-01' AND sales.date <= '2023-12-31';", 'Answer': '[(3182080,)]'}


'A total of 3,182,080 Surface Laptops were sold during 2023.'

In [10]:
run_conversation("who won the super bowl last year?")

Model's response:
None
No tool calls were made by the model.
[{'role': 'system', 'content': 'You are an assistant that help answering questions from either an AI Search Index or a CosmosDB Database. Please only answer the questions if any of the tools provide it, If tool_calls=None please state that you dont have access to the source of that information and pass the message from the function calls'}, {'role': 'user', 'content': 'who won the super bowl last year?'}, ChatCompletionMessage(content='The Los Angeles Rams won the Super Bowl LVI, which took place in February 2022. They defeated the Cincinnati Bengals to win the championship.', role='assistant', function_call=None, tool_calls=None), {'role': 'system', 'content': 'No functions or tools were called by the model.'}]
--------------------------------------------------------------------------------
Final Response:
I don't have access to the source of that information. If you need the latest information, I recommend checking a reliab

"I don't have access to the source of that information. If you need the latest information, I recommend checking a reliable sports news website or database."